<h1 style="text-align: center; color: #086A87;">Web Scrapping Python Script</h1>
<h3 style="text-align: center; color: #086A87;">By: McKenzie Kulseth</h3><br><br>
<h4 style="color: #086A87;margin:0;">Purpose of this code:</h4>
<p style="color: #086A87; margin:0;">This code will parse a webpage for data and extract that data. 
    <br><br><em>NOTE: One should orgainze the csv files being save into a directory tree to better organize the files.</p>
<p style="color: #086A87;"><br><strong>1.) Import Libraries</p>


In [1]:
import requests as req; import lxml; from bs4 import BeautifulSoup; import csv;
import numpy as np; import pandas as pd; from datetime import *; import threading as th; import os

<p style="color: #086A87;margin: 0;"> <br> <strong>2.) Get the station ids for the sounding stations from a website</strong>
<ul style="color: #086A87;margin: 0;"><li> Initialize list to ensure no station id is repeated in the list
    <li> Loop through the website in the list of websites
    <li> Use BeautifulSoup to parse the webpage for the station ids without the 'k' in front front
    <li> Append those ids to the list of station ids</ul>
</p>

In [2]:
station_ids = []
with open("stations_2.csv") as myfile:
    myreader = csv.reader(myfile,delimiter=",")
    row_count = 0
    for row in myreader:
        if row_count != 0 and len(row) != 0:
            station_ids.append(row[0])
        row_count += 1
print("There are "+str(len(station_ids))+" in the United States.")

There are 3460 in the United States.


<p style="color: #086A87;margin: 0;"> <br><strong>3.) Creating lists to hold the dates to be looped through to find sounding data.</strong>
<br> <ul style="color: #086A87;margin: 0;"><li> Get today's date to get the ending year for sounding data to search for
    <li> Create lists of years, months, and days to search for sounding data
</ul></p>

In [3]:
today = date.today()
this_year = int(today.strftime("%Y"))
years = np.arange(1990,this_year)
months = ['01','02','03','04','05','06','07','08','09','10','11','12']
print("Done creating years, months, and days to loop through for getting data.")

Done creating years, months, and days to loop through for getting data.


In [4]:
# Create a function that checks for data in website
class SoundingData:
    def __init__(self, station_id="MPX", yyyy=2020, mm=12):
        self.station_id = station_id
        self.year = str(yyyy)
        self.month = str(mm)
        self.day1 = "01"
        self.day2 = "31"
        if self.month == "02" and int(self.year)%4 == 0:
            self.day2 = "29"
        elif self.month == "02" and int(self.year)%4 != 0:
            self.day2 = "28"
        elif self.month not in ["01","03","05","07","08","10","12"]:
            self.day2 = "30"
            
        self.websites = ["http://weather.uwyo.edu/cgi-bin/sounding?region=naconf&TYPE=TEXT%3ALIST&YEAR="+self.year+"&MONTH="+self.month+"&FROM="+self.day1+"00&TO="+self.day2+"12&STNM="+self.station_id]
        
    def GetSoundingData(self):
        data_websites = []
        # Read the website(s) using bs4
        for website in self.websites:
            data_websites.append(website)

        return data_websites

In [5]:
# Test the function DataCheck
testing_Data1 = SoundingData(yyyy=2020,mm="02")
testing_DataCheck = testing_Data1.GetSoundingData()
print(testing_DataCheck)

['http://weather.uwyo.edu/cgi-bin/sounding?region=naconf&TYPE=TEXT%3ALIST&YEAR=2020&MONTH=02&FROM=0100&TO=2912&STNM=MPX']


<p style="color: #086A87;"><br><strong>4.) Loop through those dates and station ids to create a list of website to be parsed for sounding data</p>

In [6]:
print("Appending websites to a list to be looped through when finding sounding data.")
web_sites= []; count = 0
for y in np.flipud(years):
    for m in np.flipud(months):
        for ids in station_ids:
            Sounding_Data = SoundingData(ids,y,m)
            websites = Sounding_Data.GetSoundingData()
            for website in websites:
                web_sites.append(website)
print("\nDone.\nThere are "+str(len(web_sites))+" to sift through.")

Appending websites to a list to be looped through when finding sounding data.

Done.
There are 1287120 to sift through.


<p style="color: #086A87;margin: 0;"><br><strong> 5.) Get sounding data from websites</strong>
<ul style="color: #086A87;margin: 0;"><li> Loop through websites
    <li> Store sounding data in dictionary
    <li> Create a panadas dataframe to store the dictionary
    <li> Save the dataframe as a csv file
</ul>
</p>

In [7]:
for test_site in web_sites:
#     test_site = "http://weather.uwyo.edu/cgi-bin/sounding?region=naconf&TYPE=TEXT%3ALIST&YEAR=2020&MONTH=12&FROM=0100&TO=3112&STNM=BMX"
    source = req.get(test_site).text
    soup = BeautifulSoup(source,"lxml")
    data = soup.find_all("pre")
    year = ''; month = ''; station_id = '';
    if test_site[-3] != "=":
        year = test_site[-40:-36]
        month = test_site[-29:-27]
        station_id = test_site[-3:]
    else:
        year = test_site[-39:-35]
        month = test_site[-28:-26]
        station_id = test_site[-2:]
    if len(data) > 0:
        for d in range(1,len(data),2):
            d1 = data[d].text.split("\n")
            day = d1[3].split(":")[1].split("/")[0][-2:]; hour = d1[3].split(":")[1].split("/")[1];

            file_1 = station_id+"_"+year+"_"+month+"_"+day+"_"+hour+".csv"

            head = [d2 for d2 in data[d-1].text.split("\n")[2].split(" ") if d2 != '' ]
            head_1 = []
            units = [d2 for d2 in data[d-1].text.split("\n")[3].split(" ") if d2 != '' ]
            data1 = [[np.nan for z in head] for d2 in data[d-1].text.split("\n")[5:]]
            data2 = []
            count = 0
            for d1a in data[d-1].text.split("\n")[5:]:
                d1b = d1a.split(" ")
                count1 = 0
                for d1c in d1b:
                    if d1c != '':
                        data1[count][count1] = d1c
                        count1 += 1
                count += 1
            flag = [np.nan for z in head]
            while flag in data1:
                data1.remove(flag)
            count = 0
            for z in [data[d].text.split("\n")[1:-2][x].split(":") for x in range(len(data[d].text.split("\n")[1:-2]))]:
                z1 = z[0].split(" ")
                data2.append(z[1])
                flag = ""
                while flag in z1:
                    z1.remove(flag)
                tz = ""
                zz = []
                for z2 in range(len(z1)):
                    if z2 == 0:
                        tz += z1[z2]
                    else:
                        tz += "_"+z1[z2]
                head.append(tz)
                count += 1
            for z in data1:
                for k in data2:
                    z.append(k)
            
            mydf1 = pd.DataFrame(data = data1,columns=head)
            topdir = ""
            if not os.path.exists(year+"/"+month+"/"+day):
                os.makedirs(year+"/"+month+"/"+day)
                topdir = year+"/"+month+"/"+day+"/"
            else:
                topdir = year+"/"+month+"/"+day+"/"
            mydf1.to_csv(topdir+file_1,na_rep=np.nan)

KeyboardInterrupt: 